# Ejercicio 5

In [1]:
import tensorflow as tf

print("Hay GPU disponible: "),
print(tf.test.is_gpu_available())

Hay GPU disponible: 
True


# Cargar datos

Antes de empezar este ejercicio, por favor carga el zip encontrado en el repositorio de GitHub.

In [3]:
!unzip hot-dog-not-hot-dog.zip

Archive:  hot-dog-not-hot-dog.zip
   creating: hot-dog-not-hot-dog/
   creating: hot-dog-not-hot-dog/test/
   creating: hot-dog-not-hot-dog/test/hot_dog/
  inflating: hot-dog-not-hot-dog/test/hot_dog/133012.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/133015.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/133245.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/135628.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/138933.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/138937.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/138969.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/139177.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/142835.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/143012.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/144956.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/146227.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/146834.jpg  
  inflating: hot-dog-not-hot-dog/test/hot_dog/147874.jpg  
  inflating: hot-dog

In [0]:
import os

train_path = 'hot-dog-not-hot-dog/train'
test_path = 'hot-dog-not-hot-dog/test'

Las siguientes celdas crean generadores de imágenes. Prácticamente, implementan funciones que pueden ser utilizadas para obtener los datos de un directorio. 

In [7]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

img_size = 224
num_classes = 2

data_generator = ImageDataGenerator()

train_generator = data_generator.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=498,
        class_mode='categorical')

test_generator = data_generator.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        batch_size=500,
        class_mode='categorical')

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


En la siguiente celda el objetivo es cargar una red neuronal InceptionResNetV2 para clasificar imágenes. Deberán especificar weights, include_top e input_shape.

La documentación está en https://keras.io/applications/.

In [1]:
from tensorflow.python.keras.applications import InceptionResNetV2

# Crear una capa InceptionResNetV2 que se llame conv_base
conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_base.summary()

/Users/osanseviero/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/osanseviero/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/osanseviero/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/osanseviero/anaconda3/lib/python3.6/site-packages/tensorf

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
from tensorflow.python import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Flatten, Dense

model = Sequential()
model.add(conv_base)

# Implementar una capa flatten
model.add(Flatten())

# Implementar una capa densa
model.add(Dense(256, activation='relu'))

# Implementar la capa de salida
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten (Flatten)            (None, 38400)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               9830656   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 64,167,906
Trainable params: 64,107,362
Non-trainable params: 60,544
_________________________________________________________________


In [0]:
# Cómpila el modelo como en los anteriores ejercicios
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer='adam',
             metrics=['accuracy'])

In [0]:
input_x = (train_generator[0][0]/255)
input_y = (train_generator[0][1])

In [13]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('model.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

Using TensorFlow backend.


In [14]:
model.fit(input_x,
         input_y,
         epochs=20,
         validation_split=0.2,
         callbacks=[checkpoint]
         )

Train on 398 samples, validate on 100 samples
Epoch 1/20
384/398 [===========================>..] - ETA: 1s - loss: 3.6317 - acc: 0.6354
Epoch 00001: val_acc improved from -inf to 0.62000, saving model to model.hdf5
398/398 [==============================] - 64s 161ms/sample - loss: 3.6129 - acc: 0.6382 - val_loss: 59.6488 - val_acc: 0.6200
Epoch 2/20
384/398 [===========================>..] - ETA: 0s - loss: 0.7922 - acc: 0.7812
Epoch 00002: val_acc did not improve from 0.62000
398/398 [==============================] - 20s 50ms/sample - loss: 0.8298 - acc: 0.7864 - val_loss: 1982.3858 - val_acc: 0.4300
Epoch 3/20
384/398 [===========================>..] - ETA: 0s - loss: 0.9857 - acc: 0.8464
Epoch 00003: val_acc did not improve from 0.62000
398/398 [==============================] - 20s 51ms/sample - loss: 0.9661 - acc: 0.8467 - val_loss: 21.8351 - val_acc: 0.4500
Epoch 4/20
384/398 [===========================>..] - ETA: 0s - loss: 1.6767 - acc: 0.7891
Epoch 00004: val_acc did not i

In [0]:
output_x = (test_generator[0][0]/255)
output_y = test_generator[0][1]

In [16]:
model.load_weights("model.hdf5")
print(model.evaluate(output_x, output_y))

500/500 [==============================] - 7s 15ms/sample - loss: 1.2558 - acc: 0.7620
[1.2557542667388917, 0.762]
